In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.stats import sigma_clipped_stats


In [ ]:
import matplotlib.pyplot as plt
import arya

In [ ]:
import sep

In [ ]:
from astropy.nddata import CCDData

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../imaging")

from phot_utils import swap_byteorder

In [ ]:
from pathlib import Path

In [ ]:
objdir = "../yasone2/img_r_01"
fname = "nobkg.fits"
wname = "nobkg.weight.fits"
objdir = Path(objdir)

In [ ]:
img = CCDData.read(objdir / fname, unit="adu")
weights = CCDData.read(objdir / wname,  unit="adu")
cat = Table.read(objdir / "detection.cat", format="fits", hdu=2)

In [ ]:
seg = CCDData.read(objdir / "segmentation.fits", unit="adu")

In [ ]:
print(np.sum(seg.data == 0))

In [ ]:
# nx, ny = img.shape
# mask = np.zeros((nx, ny), dtype=bool)
# scale = 6
# xx, yy = np.mgrid[0:nx, 0:ny]


# for (i, src) in enumerate(cat):
#     if i % 20 == 0:
#         print(i, " / ", len(cat))
#     y0 = src["X_IMAGE"] - 1
#     x0 = src["Y_IMAGE"] - 1

#     a = scale * src["A_IMAGE"]
#     b = scale * src["B_IMAGE"]

#     theta = np.deg2rad(src["THETA_IMAGE"])

#     cos_t = np.cos(theta)
#     sin_t = np.sin(theta)

#     dx = xx - x0
#     dy = yy - y0

#     x_rot =  dx * cos_t + dy * sin_t
#     y_rot = -dx * sin_t + dy * cos_t

#     ellipse = (x_rot / a)**2 + (y_rot / b)**2 <= 1.0
#     mask |= ellipse


In [ ]:
from scipy.ndimage import distance_transform_edt

In [ ]:
plt.imshow(seg.data > 0)
# plt.scatter(cat["X_IMAGE"], cat["Y_IMAGE"], s=1)

In [ ]:
plt.imshow(mask)
plt.scatter(cat["X_IMAGE"], cat["Y_IMAGE"], s=1, alpha=0.1)

In [ ]:
mask_all = (weights.data == 0) | mask | (seg.data > 0)

In [ ]:
plt.imshow(mask_all)

In [ ]:
def sample_good_positions(mask, N):
    Nx, Ny = seg.shape
    x = np.random.randint(0, Nx, N)
    y = np.random.randint(0, Ny, N)

    good = [(mask[xx, yy] == 0)  for (yy, xx) in zip(y, x)]

    return y[good], x[good]

In [ ]:
x, y = sample_good_positions(mask_all, 30_000)

In [ ]:
global_rms = 86.8633 
thresh = 3

In [ ]:
np.sqrt(1 / np.median(weights.data))

In [ ]:
plt.imshow(img, vmin=-5*global_rms, vmax=5*global_rms)
# plt.scatter(x, y, s=0.1, lw=0, )

In [ ]:
img_data = swap_byteorder(img.data)

In [ ]:
r_ap = [1, 2, 3, 5, 7, 10, 20]

In [ ]:
flux_med = np.median(cat["FLUX_APER"], axis=0)

In [ ]:
flux_med

In [ ]:
def good_fluxes(img, x, y, r):
    flux, err, flag = sep.sum_circle(img_data, x, y, r)

    filt = (flag & (sep.OBJ_MERGED | sep.OBJ_TRUNC | sep.OBJ_SINGU | sep.APER_TRUNC | sep.APER_HASMASKED | sep.APER_ALLMASKED)) == 0
    # filt &= (flux) <  0
    return flux[filt]

In [ ]:
fluxes = [good_fluxes(img, x, y, r) for r in r_ap]

In [ ]:
np.nanstd(np.minimum(np.random.normal(size=(1000)), 0) + 5, mean=5)

In [ ]:
np.nanstd(np.random.normal(size=(1000)), mean=0)

In [ ]:
stds = [np.nanstd(flux, mean=0) for flux in fluxes]

In [ ]:
from astropy.stats import mad_std

In [ ]:
stds2 = np.array([mad_std(flux) for flux in fluxes])

In [ ]:
stds

In [ ]:
stds2 / stds

In [ ]:
sigma_clipped_stats(img.data.flatten())

In [ ]:
plt.hist((img.data[~mask_all]), bins=np.linspace(-500, 500, 50), density=True)


x_gaus = np.linspace(-400, 400, 1000)
sigma = global_rms
y_gaus = 0.9 * 1 / np.sqrt(2*np.pi)  / sigma * np.exp(-x_gaus**2 / 2 / sigma**2)
plt.plot(x_gaus, y_gaus, color=arya.COLORS[1])


In [ ]:
for i in range(len(r_ap)):
    sigma_exp = (global_rms * np.pi * r_ap[i])
    plt.hist((fluxes[i] / stds[i]), bins=np.linspace(-10, 10, 100), histtype="step", density=True)



x_gaus = np.linspace(-10, 10, 1000)
sigma = 1
y_gaus = 2 / np.sqrt(2*np.pi)  / sigma * np.exp(-x_gaus**2 / 2 / sigma**2)
plt.plot(x_gaus, y_gaus, color="k")


plt.yscale('log')
plt.ylim(1e-2, 1e1)
plt.xlim(-20, 20)
plt.xlabel("flux / bkg std")

In [ ]:
stds

In [ ]:
var_bkg_cat = cat["FLUXERR_APER"]**2 - cat["FLUX_APER"] / 2**2

In [ ]:
stds_cat = np.median(cat["FLUXERR_APER"], axis=0)

In [ ]:
np.median(np.sqrt(var_bkg_cat), axis=0)

In [ ]:
stds

In [ ]:
stds_cat

In [ ]:
np.median(np.sqrt(var_bkg_cat), axis=0)

In [ ]:
plt.plot(r_ap, (stds))
plt.plot(r_ap, (stds2))
plt.plot(r_ap, global_rms * np.array(r_ap)**2)


plt.yscale("log")

plt.xscale("log")

In [ ]:
plt.plot(r_ap, (stds2))
plt.plot(r_ap, (stds_cat))
plt.plot(r_ap, np.pi * np.array(r_ap)* global_rms, color="black")

plt.yscale("log")

plt.xscale("log")

In [ ]:
plt.plot(r_ap, (stds / stds_cat) / r_ap )
# plt.yscale("log")

# 2pt corr func

In [ ]:
img_masked = np.ma.masked_array(img.data, mask_all)

In [ ]:
# Calculate 2D autocorrelation using FFT
def autocorr_2d(image):
    """Compute 2D autocorrelation function"""
    # Remove NaNs for FFT
    img = np.nan_to_num(image, nan=0.0)
    
    # Compute autocorrelation via FFT
    f = np.fft.fft2(img)
    acf = np.fft.ifft2(f * np.conj(f)).real
    acf = np.fft.fftshift(acf)
    
    # Normalize
    acf /= acf.max()
    
    return acf


# Radial profile of the autocorrelation
def radial_profile(data, center=None):
    """Calculate radial profile"""
    if center is None:
        center = (data.shape[0] // 2, data.shape[1] // 2)
    
    y, x = np.indices(data.shape)
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    r = r.astype(int)
    
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radial_prof = tbin / nr
    
    return radial_prof



In [ ]:
acf = autocorr_2d(img_masked)
radial_acf = radial_profile(acf)


In [ ]:
# Plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 2D autocorrelation
im0 = axes[0].imshow(acf, origin='lower', cmap='RdBu_r', 
                     vmin=-0.1, vmax=1.0)
axes[0].set_title('2D Autocorrelation Function')
axes[0].set_xlabel('dx (pixels)')
axes[0].set_ylabel('dy (pixels)')
plt.colorbar(im0, ax=axes[0])

# Radial profile
pixel_scale = np.arange(len(radial_acf))
axes[1].plot(pixel_scale, radial_acf)
axes[1].axhline(0, color='k', linestyle='--', alpha=0.3)
axes[1].set_xlabel('Separation (pixels)')
axes[1].set_ylabel('Correlation')
axes[1].set_title('Radial Autocorrelation')
axes[1].set_xlim(0, min(100, len(radial_acf)//4))  # Show first quarter
axes[1].grid(True, alpha=0.3)

# Log scale to see structure better
axes[2].semilogy(pixel_scale, np.abs(radial_acf))
axes[2].set_xlabel('Separation (pixels)')
axes[2].set_ylabel('|Correlation|')
axes[2].set_title('Radial Autocorrelation (log scale)')
axes[2].set_xlim(0, min(100, len(radial_acf)//4))
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
img = CCDData.read(objdir / fname, unit="adu")
weights = CCDData.read(objdir / wname,  unit="adu")
cat = Table.read(objdir / "detection.cat", format="fits", hdu=2)

In [ ]:
seg = CCDData.read(objdir / "segmentation.fits", unit="adu")

In [ ]:
img_masked_abs = img.data.copy()
img_masked_abs[seg.data > 0] = 0

In [ ]:
plt.hist(img_masked_abs[img_masked_abs != 0] / global_rms, bins=np.linspace(-20, 20, 100), density=True)
plt.yscale("log")

x_gaus = np.linspace(-5, 5, 1000)
sigma = 1
y_gaus = 0.9 * 1 / np.sqrt(2*np.pi)  / sigma * np.exp(-x_gaus**2 / 2 / sigma**2)
plt.plot(x_gaus, y_gaus, color=arya.COLORS[1])

# plt.ylim(-1e6, 1e2)

In [ ]:
plt.imshow(img_masked_abs,vmin=-5*global_rms, vmax=5*global_rms)

In [ ]:
plt.imshow(img_masked_abs,vmin=-5*global_rms, vmax=5*global_rms)